## FAKE DATA

In [16]:
from __future__ import division
import numpy as np

zs = [-0.178654, 0.828305, 0.0592247, -0.0121089, -1.48014, 
      -0.315044, -0.324796, -0.676357, 0.16301, -0.858164]

Python's `reduce` takes the initial value in last position:

In [17]:
from functools import reduce

In [18]:
reduce(lambda c, z: c + 1, zs, 0)

10

In [19]:
import toolz
from toolz import accumulate

In [20]:
list (accumulate (lambda c, z: c + 1, zs, 0))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

We don't have a direct counterpart for Let-Over-Lambda in Python, but classes come close, though they make us manage the names manually. The `attrs` package reduces boilerplate (`dunder` methods), making OOP more tolerable.

In [21]:
from attr import attrs, attrib, Factory

In [22]:
@attrs
class StatsLol(object):
    _initial_count = attrib(init=False, default=0)
    _running_count = attrib(init=False, default=0)
    def almost_a_lambda (self, c, z):
        """not thread-safe!"""
        self._running_count = self._running_count + 1
        return self._running_count
    def run_count (self, zs):
        return reduce (self.almost_a_lambda, zs, 0)
    def all_counts (self, zs):
        return list(accumulate (self.almost_a_lambda, zs, 0))
StatsLol().all_counts(zs)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

Decouple the functionality from the means of accessing the data.

In [23]:
@attrs
class Stats(object):
    _count = attrib(init=False, default=0)
    def count (self, z):
        """not thread-safe!"""
        self._count = self._count + 1
        return self._count

list(map(Stats().count, zs))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

# RUNNING MEAN

In [24]:
@attrs
class Stats(object):
    _count = attrib(init=False, default=0)
    _mean  = attrib(init=False, default=0)
    def count (self, z):
        """not thread-safe!"""
        n  = self._count
        n1 = (n + 1)
        K  = 1.0 / n1
        self._count = n1
        x  = self._mean
        self._mean = (x + K*(z - x))
        return self._mean

list(map(Stats().count, zs))

[-0.178654,
 0.3248255,
 0.2362919,
 0.1741917,
 -0.15667464000000003,
 -0.18306953333333337,
 -0.20331617142857145,
 -0.262446275,
 -0.21517335555555556,
 -0.27947242]

In [25]:
np.mean(zs)

-0.27947241999999994

# STASH

Stuff we don't need yet, but don't want to throw away yet.

In [26]:
@attrs
class Stats(object):
    _count = attrib(init=False, default=0)
    _mean  = attrib(init=False, default=0)
    _ssqr  = attrib(init=False, default=0)
    _varc  = attrib(init=False, default=0)
    _stdv  = attrib(init=False, default=0)
    def count (self, z):
        """not thread-safe!"""
        n  = self._count
        n1 = (n + 1)
        K  = 1.0 / n1
        self._count = n1
        x  = self._mean
        self._mean = (x + K*(z - x))
        return self._mean

list(map(Stats().count, zs))

[-0.178654,
 0.3248255,
 0.2362919,
 0.1741917,
 -0.15667464000000003,
 -0.18306953333333337,
 -0.20331617142857145,
 -0.262446275,
 -0.21517335555555556,
 -0.27947242]